In [1]:
import numpy as np
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from tqdm.notebook import tqdm

# Using fixed learning rate alpha

# Using visitation table N

In [2]:
def initialize_q_table(n_states, n_actions):
    """
    Initializes a Q-table with zeros.

    Parameters:
        n_states (int): Number of states in the environment.
        n_actions (int): Number of possible actions in the environment.

    Returns:
        numpy.ndarray: Q-table of shape (n_states, n_actions) initialized with zeros.
    """
    return np.zeros((n_states, n_actions))


def initialize_visitation_table(n_states, n_actions):
    """
    Initializes a visitation count table with zeros.

    Parameters:
        n_states (int): Number of states in the environment.
        n_actions (int): Number of possible actions in the environment.

    Returns:
        numpy.ndarray: Visitation count table of shape (n_states, n_actions).
    """
    return np.zeros((n_states, n_actions))


def epsilon_greedy_policy(state, q_table, epsilon):
    """
    Selects an action using an epsilon-greedy policy.

    Parameters:
        state (int): Current state index.
        q_table (numpy.ndarray): Current Q-table.
        epsilon (float): Exploration rate (probability of choosing a random action).
    Returns:
        int: Selected action (0 to n_actions-1).
    """
    if np.random.random()<epsilon:
        return np.random.randint(0, len(q_table[state]))
    else:
        return int(np.argmax(q_table[state]))
    

def generate_episode(env, q_table, epsilon, max_steps=1000):
    """
    Generates an episode using the current Q-table and epsilon-greedy policy.

    Parameters:
        env (gym.Env): FrozenLake environment instance.
        q_table (numpy.ndarray): Current Q-table.
        epsilon (float): Exploration rate.
        max_steps (int): Maximum steps allowed per episode.

    Returns:
        list: Episode as a list of tuples (state, action, reward).
    """
    episode = []
    obs, _ = env.reset()
    for _ in range(max_steps):
        # if (eps_id+1)%5000==0:
        #     env.render()
        action = epsilon_greedy_policy(obs, q_table, epsilon)
        next_obs, reward, is_done, is_trunc, _ = env.step(action)
        episode.append([obs, action, reward])
        if is_done or is_trunc:
            break
        obs = next_obs

    return episode


def calculate_returns(episode, gamma):
    """
    Calculates the discounted return for each step in an episode.

    Parameters:
        episode (list): List of (state, action, reward) tuples.
        gamma (float): Discount factor (between 0 and 1).

    Returns:
        list: Discounted returns for each step in the episode.
    """
    returns = []
    G = 0 

    for step in reversed(episode):
        _, _, reward = step
        G = reward + gamma * G 
        returns.insert(0, G)  

    return returns


def update_q_table(q_table, n_table, episode, returns):
    """
    Updates the Q-table using the first-visit Monte Carlo method.

    Parameters:
        q_table (numpy.ndarray): Q-table to update.
        n_table (numpy.adarray): Visitation count table.
        episode (list): Episode data as list of (state, action, reward) tuples.
        returns (list): Precomputed discounted returns for each step.

    Returns:
        None (updates Q-table in-place).
    """
    visited = set()  # Track visited (state, action) pairs
    for (obs, action, _), discounted_return in zip(episode, returns):
        if (obs, action) not in visited:
            visited.add((obs, action))
            n_table[obs][action]+=1
            q_table[obs][action] += 1/n_table[obs][action] * (discounted_return - q_table[obs][action])
    
    return


def train_monte_carlo(env, num_episodes, gamma):
    """
    Executes the On-Policy Monte Carlo Control training loop with improved epsilon decay.

    Parameters:
        env (gym.Env): FrozenLake environment instance.
        num_episodes (int): Total number of training episodes.
        gamma (float): Discount factor.

    Returns:
        tuple: (Q-table, visitation table, list of total rewards per episode)
    """
    q_table = initialize_q_table(env.observation_space.n, env.action_space.n)
    n_table = initialize_visitation_table(env.observation_space.n, env.action_space.n)
    episode_rewards = []

    for eps_id in tqdm(range(num_episodes)):
        # Update epsilon using inverse decay
        epsilon = 1.0 / (eps_id/10000 + 1)
        # epsilon = 1.0 / (eps_id + 1)
        episode = generate_episode(env, q_table, epsilon, max_steps=5000)
        total_reward = sum(reward for (_, _, reward) in episode)
        episode_rewards.append(total_reward)
        discounted_returns = calculate_returns(episode, gamma)
        update_q_table(q_table, n_table, episode, discounted_returns)

        # Print progress every 5000 episodes
        if (eps_id + 1) % 5000 == 0:
            avg_reward = np.mean(episode_rewards[-5000:])
            print(f"Episode {eps_id + 1}, Average Reward (last 5000 episodes): {avg_reward:.2f}")

    return q_table, n_table, episode_rewards

In [3]:
num_episodes = 200000

env = gym.make("Taxi-v3", render_mode="rgb_array")
# env = RecordVideo(env, video_folder="./videos", episode_trigger=lambda eps: eps==num_episodes-1)
obs, _ = env.reset()
q_table, n_table, episode_rewards = train_monte_carlo(env, num_episodes, 0.99)
env.close()

  0%|          | 0/200000 [00:00<?, ?it/s]

Episode 5000, Average Reward (last 5000 episodes): -406.49
Episode 10000, Average Reward (last 5000 episodes): -83.40
Episode 15000, Average Reward (last 5000 episodes): -43.42
Episode 20000, Average Reward (last 5000 episodes): -28.51
Episode 25000, Average Reward (last 5000 episodes): -21.10
Episode 30000, Average Reward (last 5000 episodes): -15.97
Episode 35000, Average Reward (last 5000 episodes): -13.55
Episode 40000, Average Reward (last 5000 episodes): -10.74
Episode 45000, Average Reward (last 5000 episodes): -9.47
Episode 50000, Average Reward (last 5000 episodes): -6.61
Episode 55000, Average Reward (last 5000 episodes): -6.32
Episode 60000, Average Reward (last 5000 episodes): -4.56
Episode 65000, Average Reward (last 5000 episodes): -4.35
Episode 70000, Average Reward (last 5000 episodes): -3.88
Episode 75000, Average Reward (last 5000 episodes): -3.51
Episode 80000, Average Reward (last 5000 episodes): -3.45
Episode 85000, Average Reward (last 5000 episodes): -2.33
Episod

In [4]:
env = gym.make("Taxi-v3", render_mode="rgb_array")
env = RecordVideo(env, video_folder="./videos")
obs, _ = env.reset()
for _ in range(5000):
    env.render()
    action = epsilon_greedy_policy(obs, q_table, 0)
    next_obs, reward, is_done, is_trunc, _ = env.step(action)
    if is_done or is_trunc:
        break
    obs = next_obs
env.close()

c:\Users\ASUS\miniconda3\envs\rl-hands-on\Lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at e:\Reinforcement Learning\hands-on\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
